## Importing libraries

In [1]:
import pandas as pd 
import cbsodata
import json
import re
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

## Loading the data

In [2]:
# Load the data from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)

# Create a list to store the identifiers
identifiers = []

# Loop through the data and extract the Identifier for each entry
for entry in table_data:
    identifiers.append(entry['Identifier'])

# Print the list of identifiers
print(f"List of Identifiers: {identifiers}")

List of Identifiers: ['83451NED', '85928NED', '85663NED', '80072ned', '81589NED', '83147NED', '81588NED', '83149NED', '83148NED']


In [5]:
def fetch_and_save_tables(identifiers, output_folder='data'):
    """
    Fetches data for each table identifier using `cbsodata.get_data()` and saves it to a CSV file,
    skipping files that already exist.
    
    Parameters:
        identifiers (list): List of table identifiers to fetch data for.
        output_folder (str): The folder where the CSV files will be saved (default is 'data').
    
    Returns:
        None
    """
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for identifier in identifiers:
        # Define the output CSV file path
        output_file = f'{output_folder}/{identifier}.csv'
        
        # Check if the file already exists
        if os.path.exists(output_file):
            print(f"Skipping {output_file} because it already exists.")
            continue
        
        # Fetch data for the current table identifier
        print(f"Fetching data for table identifier: {identifier}")
        data = pd.DataFrame(cbsodata.get_data(identifier))
        
        # Save the DataFrame to CSV
        data.to_csv(output_file, index=False)
        
        # Print success message
        print(f"Data for {identifier} saved to {output_file}")

# Example usage
fetch_and_save_tables(identifiers)

NameError: name 'os' is not defined

In [123]:
# Function to load CSV files
def load_csv_files(file_paths):
    """
    Load multiple CSV files into DataFrames and prefix the column names with the file name.
    
    Parameters:
        file_paths (list): List of file paths to CSV files.
    
    Returns:
        list: List of DataFrames with prefixed column names.
    """
    dataframes = []
    
    for file_path in file_paths:
        # Extract the filename without the extension to use as a prefix
        file_name = file_path.split('/')[-1].split('.')[0]
        
        # Load the CSV file into a DataFrame
        df = pd.read_csv(file_path)
        
        # Prefix the column names with the filename
        df = df.add_prefix(f"{file_name}_")
        
        # Special handling for 'Perioden' and 'SBI' columns, which shouldn't be prefixed
        if f"{file_name}_Perioden" in df.columns:
            df = df.rename(columns={f"{file_name}_Perioden": 'Perioden'})  # Keep 'Perioden' without prefix
        if f"{file_name}_BedrijfstakkenBranchesSBI2008" in df.columns:
            df = df.rename(columns={f"{file_name}_BedrijfstakkenBranchesSBI2008": 'BedrijfstakkenBranchesSBI2008'})  # Keep 'BedrijfstakkenBranchesSBI2008' without prefix
        if f"{file_name}_ID" in df.columns:
            df = df.rename(columns={f"{file_name}_ID": 'ID'})  # Keep 'ID' without prefix
        if f"{file_name}_Jaar" in df.columns:
            df = df.rename(columns={f"{file_name}_Jaar": 'Jaar'})  # Keep 'Jaar' without prefix
        if f"{file_name}_Kwartaal" in df.columns:
            df = df.rename(columns={f"{file_name}_Kwartaal": 'Kwartaal'})  # Keep 'Kwartaal' without prefix
        
        # Append to the list of DataFrames
        dataframes.append(df)
    
    return dataframes

# Function to process the 'Perioden' column
def process_period_column(df, perioden_column='Perioden', year_column='Year', quarter_column='Quarter'):
    """
    Process the 'Perioden' column to extract the latest year and quarter.
    
    Parameters:
        df (DataFrame): DataFrame with 'Perioden' column.
        perioden_column (str): Name of the period column.
        year_column (str): Name of the column for extracted year.
        quarter_column (str): Name of the column for extracted quarter.
    
    Returns:
        DataFrame: DataFrame with new 'Year' and 'Quarter' columns.
    """
    # Extract latest year and quarter from 'Perioden' column
    df[[year_column, quarter_column]] = df[perioden_column].apply(lambda x: pd.Series(extract_latest_year_and_quarter(x)))
    return df

# Helper function to extract latest year and quarter
def extract_latest_year_and_quarter(period_text):
    """
    Extracts the latest year and quarter from a period that may contain a date range (e.g., "2023 april - 2024 maart").
    
    Parameters:
        period_text (str): The period string to process.
    
    Returns:
        tuple: A tuple containing the latest year and quarter.
    """
    # Regular expression to capture year and month/quarter references
    match = re.findall(r'(\d{4})\s*(\w+)', period_text)
    
    if match:
        # Take the last match in the range (latest part of the period)
        latest_year, latest_period_part = match[-1]
        latest_year = int(latest_year)
        
        # Determine the corresponding quarter for the latest period part (month or quarter)
        latest_quarter = extract_quarter(latest_period_part)
        
        return latest_year, latest_quarter
    else:
        # Return None if no valid year/quarter is found
        return None, None

# Helper function to determine the quarter based on months or quarter phrases like '3e kwartaal'
def extract_quarter(period_part):
    """
    Extracts the quarter from the period part (e.g., based on months or quarter keywords).
    
    Parameters:
        period_part (str): The part of the period string after the year.
    
    Returns:
        int: The corresponding quarter (1 to 4), or None if no valid quarter is found.
    """
    # Handle month-based quarters
    if any(month in period_part for month in ['januari', 'februari', 'maart']):
        return 1
    elif any(month in period_part for month in ['april', 'mei', 'juni']):
        return 2
    elif any(month in period_part for month in ['juli', 'augustus', 'september']):
        return 3
    elif any(month in period_part for month in ['oktober', 'november', 'december']):
        return 4
    
    # Handle phrases like '1e kwartaal', '2e kwartaal', etc.
    if '1e' in period_part:
        return 1
    elif '2e' in period_part:
        return 2
    elif '3e' in period_part:
        return 3
    elif '4e' in period_part:
        return 4
    
    # Return None if no valid quarter is found
    return None

def rename_sbi_column(df, primary_sbi_column='BedrijfstakkenBranchesSBI2008', backup_sbi_column='BedrijfskenmerkenSBI2008'):
    """
    Renames the 'BedrijfskenmerkenSBI2008' column to 'BedrijfstakkenBranchesSBI2008' if the primary column does not exist.
    
    Parameters:
        df (DataFrame): The DataFrame to process.
        primary_sbi_column (str): The desired primary SBI column name. Default is 'BedrijfstakkenBranchesSBI2008'.
        backup_sbi_column (str): The backup SBI column to rename. Default is 'BedrijfskenmerkenSBI2008'.
    
    Returns:
        DataFrame: Updated DataFrame with the renamed column.
    """
    # Check if the primary column does not exist and the backup column does exist
    if primary_sbi_column not in df.columns and backup_sbi_column in df.columns:
        # Rename 'BedrijfskenmerkenSBI2008' to 'BedrijfstakkenBranchesSBI2008'
        df = df.rename(columns={backup_sbi_column: primary_sbi_column})
        print(f"Renamed '{backup_sbi_column}' to '{primary_sbi_column}'")
    
    return df

# Function to filter by industry
def filter_by_industry(df, industry_column='BedrijfstakkenBranchesSBI2008', valid_industries=None):
    """
    Filter the DataFrame by specific industries.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        industry_column (str): Name of the industry column.
        valid_industries (list): List of valid industries to keep.
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    if valid_industries is None:
        valid_industries = [
            "Q Gezondheids- en welzijnszorg", 
            "G Handel", 
            "C Industrie", 
            "M Specialistische zakelijke diensten", 
            "N Verhuur en overige zakelijke diensten", 
            "O Openbaar bestuur en overheidsdiensten"
        ]
    return df[df[industry_column].isin(valid_industries)]

# Function to filter by year range
def filter_by_year(df, year_column='Year', start_year=2008, end_year=2022):
    """
    Filter the DataFrame by a range of years.
    
    Parameters:
        df (DataFrame): The DataFrame to filter.
        year_column (str): Name of the year column.
        start_year (int): The start year (inclusive).
        end_year (int): The end year (inclusive).
    
    Returns:
        DataFrame: Filtered DataFrame.
    """
    return df[(df[year_column] >= start_year) & (df[year_column] <= end_year)]

def clean_dataframe(df, id_column='ID', year_column='Year', quarter_column='Quarter', sbi_column='BedrijfstakkenBranchesSBI2008', sick_leave_column='80072ned_Ziekteverzuimpercentage_1', columns_to_drop=None):
    """
    Cleans the DataFrame by reordering columns to have 'Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008', and '80072ned_Ziekteverzuimpercentage_1' as the first four columns,
    dropping specified columns, and removing duplicate rows.
    
    Parameters:
        df (DataFrame): The DataFrame to clean.
        year_column (str): The year column name. Default is 'Year'.
        quarter_column (str): The quarter column name. Default is 'Quarter'.
        sbi_column (str): The column name for SBI. Default is 'BedrijfstakkenBranchesSBI2008'.
        sick_leave_column (str): The sick leave column name. Default is '80072ned_Ziekteverzuimpercentage_1'.
        columns_to_drop (list): List of column names to drop. Default is ['ID', 'Perioden'].
    
    Returns:
        DataFrame: Cleaned DataFrame with reordered columns, specified columns dropped, and duplicates removed.
    """
    if columns_to_drop is None:
        columns_to_drop = ['ID', 'Perioden', 'Jaar', 'Kwartaal']

    # Define the preferred order of the first four columns
    preferred_columns = [year_column, quarter_column, sbi_column, sick_leave_column]

    # Collect remaining columns in their original order (excluding the preferred ones)
    remaining_columns = [col for col in df.columns if col not in preferred_columns]

    # Combine the preferred columns with the remaining ones
    ordered_columns = preferred_columns + remaining_columns

    # Reorder DataFrame
    df = df[ordered_columns]

    # Drop the specified columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Remove duplicate rows
    df = df.drop_duplicates()

    return df

def group_and_sum(df, group_by_columns):
    """
    Group the DataFrame by specified columns and sum the numeric values.
    
    Parameters:
        df (DataFrame): The DataFrame to group.
        group_by_columns (list): List of columns to group by (e.g., ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008']).
    
    Returns:
        DataFrame: Grouped and summed DataFrame.
    """
    # Group by the specified columns and sum the numeric columns
    df = df.groupby(group_by_columns).sum(numeric_only=True).reset_index()
    
    return df

def process_tables(tables):
    """
    Process and combine multiple tables into a single DataFrame by:
    - Processing the 'Perioden' column to extract 'Year' and 'Quarter'
    - Filtering by specific industries and year range
    - Reordering columns to have 'Year' and 'Quarter' after 'ID'
    - Dropping unnecessary columns
    - Removing duplicates
    - Grouping by 'Year', 'Quarter', and 'BedrijfstakkenBranchesSBI2008' and summing the data
    
    Parameters:
        tables (list): List of DataFrames.
    
    Returns:
        DataFrame: The final combined DataFrame.
    """
    # Concatenate the list of tables into one DataFrame
    concat_df = pd.concat(tables, ignore_index=True)
    
    # Step 1: Process the 'Perioden' column
    period_df = process_period_column(concat_df)
    
    # Step 2: Fill missing 'BedrijfstakkenBranchesSBI2008' with 'BedrijfskenmerkenSBI2008'
    sbi_df = rename_sbi_column(period_df)
    
    # Step 3: Filter by industry
    industry_df = filter_by_industry(sbi_df)
    
    # Step 4: Filter by year range (2008 to 2022)
    year_df = filter_by_year(industry_df)
    
    # Step 5: Clean the DataFrame: reorder columns, drop unnecessary ones, and remove duplicates
    clean_df = clean_dataframe(year_df)
    
    # Step 6: Group by 'Year', 'Quarter', and 'BedrijfstakkenBranchesSBI2008' and sum the numeric columns
    final_df = group_and_sum(clean_df, ['Year', 'Quarter', 'BedrijfstakkenBranchesSBI2008'])
    
    return final_df

# Load the identifiers from the JSON file
with open('data/table_selection.json', 'r', encoding='utf-8') as json_file:
    table_data = json.load(json_file)
    identifiers = [entry['Identifier'] for entry in table_data]

# Dynamically generate the file paths using the identifiers
file_paths = [f'data/{identifier}.csv' for identifier in identifiers]

# Load and process the data
tables = load_csv_files(file_paths)
final_df = process_tables(tables)

# Save the final DataFrame to a CSV file
final_df.to_csv('data/merged_tables.csv', index=False)